<a href="https://colab.research.google.com/github/FritscheLab/EPID731/blob/main/Day2/EPID731_Accessing_EHR_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EPID731 - Accessing EHR Data

## Before you start: Create a copy of this notebook

1. Go to the `File` menu in the top left corner and select `Save a copy in  Drive` (If you can't see the File menu, go to the top right and press the **&bigvee;** button.
2. Close the original file `EPID731_Accessing_EHR_Data.ipynb` and follow the exercise in `Copy of EPID731_Accessing_EHR_Data.ipynb`

## Exercise Overview: Identifying Individuals with Concussion and Aspirin Use

This exercise involves working with the Eunomia database, a simulated medical dataset. Our goal is to identify individuals who experienced a concussion and subsequently took aspirin within a month of the injury. We will approach this through two methodologies:

1. **Data Combination in R**: We will download individual datasets, merge them in R, and apply filters to extract the necessary information.
2. **Single SQL Query**: We will construct and execute a SQL query that performs the data combination and filtering directly on the database server.

### Learning Objectives

- Develop proficiency in querying databases using R.
- Understand the techniques for merging data from multiple tables.
- Learn to apply filters and manipulate data using both R and SQL.


## Part 1: Setting Up the Environment

### Configuring the Google Colab Environment

Execute the following steps to prepare your Google Colab environment for the tasks ahead:

In [ ]:
# Set environment variables
Sys.setenv(JAVA_HOME = '/usr/lib/jvm/java-11-openjdk-amd64')
Sys.setenv(PATH = paste(Sys.getenv("PATH"), "/usr/lib/jvm/java-11-openjdk-amd64/bin", sep = ":"))
Sys.setenv(LD_LIBRARY_PATH = paste(Sys.getenv("LD_LIBRARY_PATH"), "/usr/lib/jvm/java-11-openjdk-amd64/lib/server", sep = ":"))

# Check if symbolic link for libjvm.so exists, if not, create it
if (!file.exists("/usr/lib/libjvm.so")) {
    system2("ln", c("-s", "/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so", "/usr/lib/libjvm.so"), stdout = TRUE, stderr = TRUE)
}

# Run javareconf to ensure R is configured to use the specified Java installation
system2('R', 'CMD javareconf', stdout = TRUE, stderr = TRUE)

### Installing and Loading Required Libraries

Ensure all necessary R libraries are installed and loaded. This process may take ~5 minutes:

In [ ]:
# Install rJava
install.packages("rJava")

# Install other required packages
install.packages(c("remotes", "dplyr", "lubridate", "knitr", "DatabaseConnector",
  "plogr", "RSQLite", "triebeard", "SqlRender", "urltools", "checkmate"))
remotes::install_github("ohdsi/Eunomia", ref = "v1.0.0")

# Load libraries
library(rJava)
library(DatabaseConnector)
library(dplyr)
library(lubridate)
library(knitr)

### Establishing a Database Connection

We will establish a connection to the local Eunomia database:

In [ ]:
# Establish a connection to the Eunomia database
connectionDetails <- Eunomia::getEunomiaConnectionDetails()
connection <- connect(connectionDetails)

# List available tables in the database
tables <- dbListTables(connection)
cat("Available tables in the database:\n")
print(tables)

### Additional Descriptions for Database Tables:

Quick overview of the tables available in the Eunomia dataset:

| Table Name               | Description                                                                 |
|--------------------------|-----------------------------------------------------------------------------|
| `care_site`              | Information about care sites like hospitals and clinics.                    |
| `cdm_source`             | Source details of the CDM data.                                             |
| `cohort`                 | Defines groups of persons for observational studies.                        |
| `cohort_attribute`       | Attributes associated with specific cohorts.                                |
| **`concept`**                | **Standardized medical concepts.**                                          |
| `concept_ancestor`       | Hierarchical relationships between concepts.                                |
| `concept_class`          | Classification of concepts.                                                 |
| `concept_relationship`   | Relationships between different concepts.                                   |
| `concept_synonym`        | Synonyms for medical concepts.                                              |
| `condition_era`          | Aggregated periods of condition occurrences.                                |
| **`condition_occurrence`**   | **Individual instances of conditions.**                                     |
| `cost`                   | Financial data related to healthcare events.                                |
| `death`                  | Records of death details.                                                   |
| `device_exposure`        | Records of device usage in patient care.                                    |
| `domain`                 | Various domains of medical concepts.                                        |
| `dose_era`               | Periods of consistent dosages of medication.                                |
| `drug_era`               | Aggregated periods of drug exposures.                                       |
| **`drug_exposure`**          | **Individual instances of drug usage.**                                     |
| `drug_strength`          | Strength details of drugs.                                                  |
| `fact_relationship`      | Relationships between various facts in the dataset.                         |
| `location`               | Geographical information of persons.                                        |
| `measurement`            | Records of measurements taken, such as lab results.                         |
| `metadata`               | Metadata related to the dataset.                                            |
| `note`                   | Unstructured clinical notes.                                                |
| `note_nlp`               | NLP-processed clinical notes.                                               |
| `observation`            | Observational data not classified as physical exams.                        |
| `observation_period`     | Periods during which observations were made.                                |
| `payer_plan_period`      | Periods of insurance coverage for patients.                                 |
| **`person`**                 | **Demographic details of individuals in the dataset.**                      |
| `procedure_occurrence`   | Records of medical procedures performed.                                    |
| `provider`               | Information about healthcare providers.                                     |
| `relationship`           | Relationships between various medical concepts.                             |
| `source_to_concept_map`  | Mapping from source codes to standard concepts.                             |
| `specimen`               | Records of biological specimens.                                            |
| `visit_detail`           | Detailed records of individual visits.                                      |
| `visit_occurrence`       | Records of visit occurrences.                                               |
| **`vocabulary`**            | **Information about the standardized vocabulary used in the dataset.**      |

### Displaying Top 5 Entries from Key Tables

To better understand the data we will be working with, it is crucial to examine samples from key tables in the Eunomia database. Below, we provide R code to query and display the top 5 entries from several relevant tables. This step will help you visualize the type of data available and practice querying with R.

#### R Code to Query and Display Data


In [ ]:
# Function to query and print the top 5 entries of a specified table
print_top_5_entries <- function(table_name, connection) {
  cat("\n### Top 5 entries from the", table_name, "table:\n")

  # Construct the SQL query to retrieve the top 5 entries
  query <- sprintf("SELECT * FROM %s LIMIT 5;", table_name)

  # Execute the query and fetch the results
  data <- dbGetQuery(connection, query)

  # Display the data in a user-friendly format
  if (nrow(data) == 0) {
    cat("No data available in this table.\n")
  } else {
    print(kable(data, format = "markdown"))
  }
}

# List of tables for which we want to display the top 5 entries
tables_to_use <- c("concept", "condition_occurrence", "drug_exposure", "person", "vocabulary")

# Execute the function for each table and print the results
for (table in tables_to_use) {
  print_top_5_entries(table, connection)
}

### Practical Tips

- **Ensure Database Connection**: Before running the code, verify that the `connection` object is correctly established to avoid connection errors.
- **Handling Different Data**: Depending on the structure and size of your data tables, you may need to adjust the SQL query to handle specific data types or constraints.


## Part 2: Method 1 - Combining Data in R

In this section, we will demonstrate how to connect to and interact with a SQL database using R. This involves establishing a database connection, querying data, and subsequently combining these datasets for further analysis.

### Establishing a Database Connection and Querying the Database

First, we establish a connection to the Eunomia database ("Establishing a Database Connection" above).

Once the connection is set up, we can proceed to query data from the database. For our exercise, we are particularly interested in individuals who experienced a concussion and took aspirin within a month of their concussion.



#### Retrieve Person Data

Let's start by retrieving relevant data from the `person` table.

In [ ]:
# Reuse the connection setup code
connectionDetails <- Eunomia::getEunomiaConnectionDetails()
connection <- connect(connectionDetails)

# SQL query to fetch data from the person table
query_person <- "
  SELECT person_id, birth_datetime, gender_source_value
  FROM person;"
person_data <- dbGetQuery(connection, query_person)
head(person_data)  # Display the first few rows of the person data

#### Retrieve Concussion and Drug Exposure Data

Next, we fetch data regarding concussion occurrences and aspirin use.


In [ ]:
# SQL query to fetch data on concussions
query_concussion <- "
  SELECT co.person_id, co.condition_start_date, co.condition_end_date,
         c.concept_name AS condition_name
  FROM condition_occurrence AS co
  JOIN concept AS c ON co.condition_concept_id = c.concept_id
  WHERE LOWER(c.concept_name) LIKE '%concussion%';"
concussion_data <- dbGetQuery(connection, query_concussion)
head(concussion_data)  # Display concussion data

# SQL query to fetch data on aspirin use
query_aspirin <- "
  SELECT de.person_id, de.drug_exposure_start_date, de.drug_exposure_end_date,
         c.concept_name AS drug_name
  FROM drug_exposure AS de
  JOIN concept AS c ON de.drug_concept_id = c.concept_id
  WHERE LOWER(c.concept_name) LIKE '%aspirin%';"
aspirin_data <- dbGetQuery(connection, query_aspirin)
head(aspirin_data)  # Display aspirin data

### Combining Data in R

With the data retrieved, we can now combine the datasets to find individuals who fit our criteria of having taken aspirin within 30 days following a concussion.


In [ ]:
# Merging concussion and person data
concussion_person_data <- merge(concussion_data, person_data, by = "person_id")

# Merging the above with aspirin data
full_data <- merge(concussion_person_data, aspirin_data, by = "person_id")

# Filtering to find relevant cases
relevant_cases <- full_data[
  full_data$drug_exposure_start_date <= full_data$condition_start_date + 30 &
  full_data$drug_exposure_end_date >= full_data$condition_start_date, ]


head(relevant_cases)  # Display the relevant cases

## Part 3: Method 2 - Combining Data in SQL

In this part, we will utilize SQL to perform data combination and filtering directly within the database, which can be more efficient for large datasets. This method reduces data transfer between the database and the application layer and takes advantage of the database's optimized query processing capabilities.

### Step 1: SQL Query to Fetch Data

We will construct a single SQL query that joins multiple tables.

In [ ]:
query_concussion_person_drug <- "
  SELECT co.person_id, co.condition_start_date, co.condition_end_date,
    c.concept_name AS condition_name, p.birth_datetime, p.gender_source_value,
    de.drug_exposure_start_date, de.drug_exposure_end_date,
    c2.concept_name AS drug_name
  FROM condition_occurrence AS co
  JOIN concept AS c ON co.condition_concept_id = c.concept_id
  JOIN person AS p ON co.person_id = p.person_id
  LEFT JOIN drug_exposure AS de ON co.person_id = de.person_id
  LEFT JOIN concept AS c2 ON de.drug_concept_id = c2.concept_id
  WHERE LOWER(c.concept_name) LIKE '%concussion%' AND
    LOWER(c2.concept_name) LIKE '%aspirin%';
"
concussion_person_drug_data <- dbGetQuery(connection, query_concussion_person_drug)
head(concussion_person_drug_data)

### Step 2: Filtering for Overlap in R

Next, apply the overlap conditions using base R functions:

In [ ]:
# Apply the filtering conditions using base R
overlap_filtered <- concussion_person_drug_data[
  concussion_person_drug_data$drug_exposure_start_date <= concussion_person_drug_data$condition_start_date + 30 &
  concussion_person_drug_data$drug_exposure_end_date >= concussion_person_drug_data$condition_start_date, ]

# Display the filtered results
head(overlap_filtered)

### Disconnecting from the Database

Finally, disconnect from the database.

In [ ]:
disconnect(connection)


## Conclusion

In this exercise, we've explored two methods for identifying individuals who had a concussion and took aspirin within a month of the concussion using the Eunomia database. We've learned to:

- Connect to a SQL-based database using R.
- Query and manipulate the extracted data in R.
- Use SQL to perform complex queries directly on the database.

By comparing these two methods, we understand the trade-offs between pulling data into R for manipulation versus performing the manipulations directly in the database.




## Feel free to explore the dataset further:

- Can you find individuals who took "Acetaminophen" within a month of a concussion?
- Can you find individuals with a "Suture open wound" procedure data within a year before a concussion?

## Extra Assignment: Extracting Unique Drug Concepts from the Eunomia Database

To prepare for the lab exercise on Thursday, we will extract all unique drug concepts from the Eunomia database and save the results to a CSV file. This data will be used to showcase how to harmonize and classify medications using a GPT API, while ensuring no PHI (Protected Health Information) is exported.

Here are the steps for this task:

1. **Query to Extract Unique Drug Concepts from the Eunomia database**:  
   We will use the following SQL query to extract unique drug concepts from the `drug_exposure` table by joining it with the `concept` table.
2. **Save and Store the Extracted Data:**  
Save the extracted data to a CSV file and provide the file path for future reference.

In [ ]:
%%R
# Establish a connection to the Eunomia database
connectionDetails <- Eunomia::getEunomiaConnectionDetails()
connection <- connect(connectionDetails)

# Define the query to extract unique drug concepts
query_drugs <- "
  SELECT DISTINCT
    de.drug_concept_id,
    c.concept_name,
    c.concept_code,
    c.vocabulary_id AS source
  FROM
    drug_exposure de
  JOIN
    concept c ON de.drug_concept_id = c.concept_id
  ORDER BY
    c.concept_name;"

# Execute the query and store the results in drug_data
drug_data <- dbGetQuery(connection, query_drugs)
head(drug_data)

# Save the concept data to a CSV file
file_path <- "unique_drug_concepts.csv"
write.csv(drug_data, file_path, row.names = FALSE)
cat("The data of the unique drug concepts have been saved to:", file_path)

# Save the unique medication concept names to a text file
file_path <- "unique_drug_concept_names.txt"
write(drug_data$concept_name, file_path)
cat("\nThe unique drug concepts names have been saved to:", file_path)

# Disconnect from the database
disconnect(connection)